In [26]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from torch.utils.data import  Dataset 
import torchvision.transforms as transforms
from PIL import Image
from typing import Tuple, Dict 
import matplotlib.pyplot as plt


In [27]:
class CFG:
    experiment_id           = 'LG38'
    model_name              = 'UNet'
    train_bs                = 8
    valid_bs                = 8
    crop_size               = (64 , 64)
    num_crops               = 1
    epochs                  = 3
    lr                      = 0.0001
    data_train_test_split   = 0.9
    data_train_val_split    = 0.8
    device                  = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    mask_train              = False
    cancerous_train         = True
    hard_to_classify_train  = False
    inflammatory_train      = False
    stroma_train            = False
    
enabled_masks = {
    "cancerous": CFG.cancerous_train,
    "hard_to_classify": CFG.hard_to_classify_train,
    "inflammatory": CFG.inflammatory_train,
    "stroma": CFG.stroma_train
}
class_names = [name for name, enabled in enabled_masks.items() if enabled]
num_masks = sum([CFG.mask_train, CFG.cancerous_train, CFG.hard_to_classify_train, CFG.inflammatory_train, CFG.stroma_train])

In [28]:
def is_kaggle() -> bool:
    return os.path.exists('/kaggle/')

def is_notebook() -> bool:

    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell':
            return True  
        elif shell == 'TerminalInteractiveShell':
            return False  
        else:
            return False  
    except NameError:
        return False  

if is_kaggle():
    dataset_path = '/kaggle/input/'
    reports_dir = '/kaggle/working/'
    models_dir = '/kaggle/working/'
else:
    if is_notebook():
        os.makedirs(os.path.join(os.path.dirname(os.getcwd()), 'reports'), exist_ok=True) 
        dataset_path = os.path.join(os.path.dirname(os.getcwd()), 'data')        
        reports_dir = os.path.join(os.path.dirname(os.getcwd()), 'reports')
        models_dir = os.path.join(os.path.dirname(os.getcwd()), 'models')
        
    else:
        os.makedirs(os.path.join(os.path.dirname(os.getcwd()), 'capstrone_group5','src', 'reports'), exist_ok=True) 
        dataset_path = os.path.join(os.path.dirname(os.getcwd()), 'capstrone_group5','src', 'data')
        reports_dir = os.path.join(os.path.dirname(os.getcwd()), 'capstone_group5', 'src', 'reports')
        models_dir = os.path.join(os.path.dirname(os.getcwd()), 'capstone_group5', 'src', 'models')

In [ ]:
if torch.cuda.is_available():
    print(f"Current device index: {torch.cuda.current_device()}")
    print(f"Device name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("CUDA is not available. Check your PyTorch installation and GPU drivers.")
allocated_memory = torch.cuda.memory_allocated()
GPU_used = round(allocated_memory / 1024**3, 2)

In [ ]:
print(f"Number of active masks is {num_masks}.\n")
print(f"Dataset path: {dataset_path}")
print(f"Dataset path: {reports_dir}")
print(f"Dataset path: {models_dir}\n")

print(f"CUDA available: {torch.cuda.is_available()}\n")
print(f"Number of GPUs: {torch.cuda.device_count()}")
print(f"GPU Memory Allocated: {allocated_memory / 1024**3:.2f} GB")

###  CREATING DATASET

In [31]:
class OriginalImageDataset(Dataset):
    def __init__(self, root_dir: str, mask_config: Dict[str, bool]):
        """
        Args:
            root_dir (str): Directory with images and masks.
            mask_config (dict): Dictionary where keys are mask names and values are True/False.
        """
        self.root_dir = root_dir
        self.mask_config = mask_config

        self.image_paths = []
        self.mask_paths = {key: [] for key in mask_config.keys() if mask_config[key]}

        for subdir, dirs, files in os.walk(root_dir):
            for file in files:
                if file.endswith("ROI.png"):
                    image_path = os.path.join(subdir, file)
                    mask_base_path = image_path.replace("ROI.png", "")

                    for mask_type in self.mask_config.keys():
                        if self.mask_config[mask_type]:
                            mask_path = mask_base_path + mask_type.upper() + ".png"
                            if os.path.exists(mask_path):
                                self.mask_paths[mask_type].append(mask_path)
                            else:
                                raise FileNotFoundError(f"Mask path {mask_path} does not exist.")
                    
                    self.image_paths.append(image_path)

    def __len__(self) -> int:
        return len(self.image_paths)

    def __getitem__(self, idx: int) -> Tuple[Image.Image, Dict[str, Image.Image]]:
        """
        Args:
            idx (int): Index for data retrieval.

        Returns:
            Tuple[Image.Image, Dict[str, Image.Image]]: The original image and its corresponding masks.
        """
        # Load image
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")

        # Load masks
        masks = {}
        for mask_type in self.mask_config.keys():
            if self.mask_config[mask_type]:
                mask_path = self.mask_paths[mask_type][idx]
                mask = Image.open(mask_path).convert("L")
                masks[mask_type] = mask

        # Return the original image and its masks
        return image, masks

In [32]:
transform = transforms.Compose([
    #transforms.Resize(CFG.img_size),
    transforms.ToTensor(),
])

mask_config = {
    "cancerous": CFG.cancerous_train,
    "hard_to_classify": CFG.hard_to_classify_train,
    "inflammatory": CFG.inflammatory_train,
    "stroma": CFG.stroma_train,
    "mask": CFG.mask_train
}

In [33]:
dataset_original = OriginalImageDataset(root_dir=dataset_path, mask_config=mask_config)
image, masks = dataset_original[0]

# Analysis of the photo size

In [ ]:
widths = []
heights = []

for i in range(len(dataset_original)):
    image, _ = dataset_original[i]  # Assuming dataset[i] returns (image, masks)
    
    if not isinstance(image, torch.Tensor):
        image = transforms.ToTensor()(image)
    
    _, h, w = image.shape  # Shape is typically (C, H, W)
    widths.append(w)
    heights.append(h)


# Scatter plot of image widths vs heights
plt.figure(figsize=(10, 10))
plt.scatter(widths, heights, alpha=0.5)
plt.title("Scatter Plot of Image Widths vs Heights")
plt.xlabel("Width")
plt.ylabel("Height")
plt.grid(True)
plt.show()

# Histogram of image widths
plt.figure(figsize=(10, 5))
plt.hist(widths, bins=30, alpha=0.7, color='blue')
plt.title("Histogram of Image Widths")
plt.xlabel("Width")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

# Histogram of image heights
plt.figure(figsize=(10, 5))
plt.hist(heights, bins=30, alpha=0.7, color='green')
plt.title("Histogram of Image Heights")
plt.xlabel("Height")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

In [ ]:
print(f"Median widths: {np.median(widths)}")
print(f"Median heights: {np.median(heights)}")
print(f"Mean widths: {np.mean(widths)}")
print(f"Mean heights: {np.mean(heights)}")

In [ ]:
def calculate_iqr(data):
    q75, q25 = np.percentile(data, [75, 25])  
    return q75 - q25

widths_iqr = calculate_iqr(widths)
heights_iqr = calculate_iqr(heights)

# Output results
print(f"Widths IQR: {widths_iqr}")
print(f"Heights IQR: {heights_iqr}")

**CONCLUSIONS**
 
1. Median width is 931 and median heights is 913, with a mean value being slightly higher. 
2. IQR: the middle 50% of the widths (between the 25th and 75th percentile) is spread accross a range of 738 pixels. This indicates rather high level of variation in the widths data, there are likely some higher outliers in the dataset.
3. IQR: the middle 50% of the heights data is spread acrross 656 pixels, this indicates as well rather high level of variation in the heights.
4. For both width and height the distribution of the size is left-skewed, majority of the images does not exceeds 1000 px.
5. By including the images with too much size  (e.x.64x64), the picture is not sufficiently covered covered by the kernel convolution function.
6. The minimum size necessary for this project is 256x256 as a limit for the available computation power, however the images still offer possibility to extend the input image size.